In [1]:
## PackedSequence 와 PaddedSequence

이 튜토리얼에서는 RNN / LSTM 계열의 모델에서 sequence batch를 잘 활용할 수 있는 PackedSequence 와 PaddedSequence를 만드는 법을 배워보겠습니다.

PyTorch 라이브러리 안에는 다음 4가지 함수들이 주어집니다.

pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

하지만 함수 이름만 봐서는 상당히 헷갈릴 수 있기 때문에 다음 그림을 참고하시면 이해하기 편하실 것 같습니다.

In [2]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

In [3]:
# Random word from random word generator
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

# Make dictionary
char_set = ['<pad>'] + list(set(char for seq in data for char in seq)) # Get all characters and include pad token
char2idx = {char: idx for idx, char in enumerate(char_set)} # Constuct character to index dictionary
print('char_set:', char_set)
print('char_set length:', len(char_set))

char_set: ['<pad>', 'l', 's', 'w', 'u', 'g', 'o', 'c', 'r', 'i', 'm', 't', 'e', 'h', 'p', 'a', ' ', 'n', 'd']
char_set length: 19


In [4]:
# Convert character to index and make list of tensors
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

# Check converted result
for sequence in X:
    print(sequence)

tensor([13, 12,  1,  1,  6, 16,  3,  6,  8,  1, 18])
tensor([10,  9, 18, 17,  9,  5, 13, 11])
tensor([ 7, 15,  1,  7,  4,  1, 15, 11,  9,  6, 17])
tensor([14, 15, 11, 13])
tensor([ 2, 13,  6,  8, 11, 16,  7,  9,  8,  7,  4,  9, 11])


In [5]:
# Make length tensor (will be used later in 'pack_padded_sequence' function)
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

lengths: [11, 8, 11, 4, 13]


In [6]:
# Make a Tensor of shape (Batch x Maximum_Sequence_Length)
padded_sequence = pad_sequence(X, batch_first=True) # X is now padded sequence
print(padded_sequence)
print(padded_sequence.shape)

tensor([[13, 12,  1,  1,  6, 16,  3,  6,  8,  1, 18,  0,  0],
        [10,  9, 18, 17,  9,  5, 13, 11,  0,  0,  0,  0,  0],
        [ 7, 15,  1,  7,  4,  1, 15, 11,  9,  6, 17,  0,  0],
        [14, 15, 11, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 2, 13,  6,  8, 11, 16,  7,  9,  8,  7,  4,  9, 11]])
torch.Size([5, 13])


In [7]:
# Sort by descending lengths
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
sorted_X = [X[idx] for idx in sorted_idx]

# Check converted result
for sequence in sorted_X:
    print(sequence)

tensor([ 2, 13,  6,  8, 11, 16,  7,  9,  8,  7,  4,  9, 11])
tensor([13, 12,  1,  1,  6, 16,  3,  6,  8,  1, 18])
tensor([ 7, 15,  1,  7,  4,  1, 15, 11,  9,  6, 17])
tensor([10,  9, 18, 17,  9,  5, 13, 11])
tensor([14, 15, 11, 13])


In [8]:
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence)

PackedSequence(data=tensor([ 2, 13,  7, 10, 14, 13, 12, 15,  9, 15,  6,  1,  1, 18, 11,  8,  1,  7,
        17, 13, 11,  6,  4,  9, 16, 16,  1,  5,  7,  3, 15, 13,  9,  6, 11, 11,
         8,  8,  9,  7,  1,  6,  4, 18, 17,  9, 11]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)


In [9]:
# one-hot embedding using PaddedSequence
eye = torch.eye(len(char_set)) # Identity matrix of shape (len(char_set), len(char_set))
embedded_tensor = eye[padded_sequence]
print(embedded_tensor.shape) # shape: (Batch_size, max_sequence_length, number_of_input_tokens)

torch.Size([5, 13, 19])


In [10]:
# one-hot embedding using PackedSequence
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

torch.Size([47, 19])


In [ ]:
# declare RNN
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

In [ ]:
rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape) # shape: (batch_size, max_seq_length, hidden_size)
print(hidden.shape)     # shape: (num_layers * num_directions, batch_size, hidden_size)

In [ ]:
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

In [ ]:
unpacked_sequence, seq_lengths = pad_packed_sequence(embedded_packed_seq, batch_first=True)
print(unpacked_sequence.shape)
print(seq_lengths)

In [ ]:
embedded_padded_sequence = eye[pad_sequence(sorted_X, batch_first=True)]
print(embedded_padded_sequence.shape)

In [ ]:
sorted_lengths = sorted(lengths, reverse=True)
new_packed_sequence = pack_padded_sequence(embedded_padded_sequence, sorted_lengths, batch_first=True)
print(new_packed_sequence.data.shape)
print(new_packed_sequence.batch_sizes)